In [1]:
import requests

url = "https://public-api.birdeye.so/defi/v2/tokens/new_listing?limit=10&meme_platform_enabled=false"

headers = {
    "accept": "application/json",
    "x-chain": "solana",
    "X-API-KEY": "f62acc1758d747679351d6f8c24ff370"
}

response = requests.get(url, headers=headers)

print(response.text)

{"success":true,"data":{"items":[{"address":"C3f7Fsxx1cDrNhs3rCQhiPxjphuxckCE6xgicik2pump","symbol":"MRSTREAM","name":"Mr Stream","decimals":6,"source":"pump_amm","liquidityAddedAt":"2025-09-15T10:11:14","logoURI":"https://ipfs.io/ipfs/QmP7ohfNKyvLfxH3FGqp8A6krRs6gMhhbav5fH7iqmeYCS","liquidity":39834.79732539112},{"address":"2HyLEhnKykQhZLVbcpTwMyiMSngE5RywsscaC7z5pump","symbol":"CAMEO","name":"CameoCoin","decimals":6,"source":"pump_amm","liquidityAddedAt":"2025-09-15T10:11:02","logoURI":null,"liquidity":39830.61188149075},{"address":"GqAdNrhpVMnYD3XBrZadnxS85tai3HBWKEzeabwoAZNA","symbol":"DWRF","name":"Dancing Walking Rearranging Furn","decimals":6,"source":"meteora_dynamic_bonding_curve","liquidityAddedAt":"2025-09-15T10:10:55","logoURI":"https://images.pump.fun/coin-image/6wjKRiQyoTw2ToErbSLxWcZPMkmnnyeHAjLeRgsqpump?variant=86x86&ipfs=bafkreid22rpzy5k25zf67mrchyrog3b73jma7yrn567on6gmrvo6ryopia&src=https%3A%2F%2Fipfs.io%2Fipfs%2Fbafkreid22rpzy5k25zf67mrchyrog3b73jma7yrn567on6gmrvo6ry

In [6]:
import requests
import os
from dotenv import load_dotenv

# Load .env file
load_dotenv()

# Fetch API key from .env (make sure your .env has API_KEY=xxxxx)
API_KEY = os.getenv("API_KEY")

url = "https://public-api.birdeye.so/defi/v2/tokens/new_listing"

params = {
    "limit": 10,
    "meme_platform_enabled": "false"
}

headers = {
    "accept": "application/json",
    "x-chain": "solana",
    "x-api-key": API_KEY   # ✅ use the variable we defined
}

response = requests.get(url, headers=headers, params=params)

print(response.status_code)
print(response.json())




ConnectionError: HTTPSConnectionPool(host='public-api.birdeye.so', port=443): Max retries exceeded with url: /defi/v2/tokens/new_listing?limit=10&meme_platform_enabled=false (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001EE23C42850>: Failed to resolve 'public-api.birdeye.so' ([Errno 11001] getaddrinfo failed)"))